In [2]:
# Importing libraries
import pandas as pd
import os
import numpy as np
np.set_printoptions(precision=4)
import catboost
from catboost import *
from catboost import datasets
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from scipy.stats import boxcox
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
import shap
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [1]:
# Set up figures' format
%config InlineBackend.figure_format = 'retina'

In [1]:
# CatBoost check-up
print(catboost.__version__)
!python --version

1.2.3
Python 3.9.18


In [3]:
# Reading input files
syd,preds = pd.read_csv('syd_17.csv'), pd.read_csv('syd_17_preds.csv')

In [4]:
# Separating income and id columns
income = syd['income']
id = preds['id']

In [5]:
id

0        505
1        632
2        432
3        798
4       1667
        ... 
9411    9205
9412    8959
9413    9670
9414    9269
9415    9690
Name: id, Length: 9416, dtype: int64

In [6]:
income

0         31
1        110
2        176
3        240
4        380
        ... 
9411    4513
9412    4523
9413    4596
9414    4666
9415    4750
Name: income, Length: 9416, dtype: int64

In [7]:
# Normalising predictor variables
first_col = syd.iloc[:, 0]
remaining_cols = syd.iloc[:, 1:]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the remaining columns of X and transform them
remaining_scaled = pd.DataFrame(scaler.fit_transform(remaining_cols), columns=remaining_cols.columns)

# Reset indices to ensure they align for concatenation
first_col = first_col.reset_index(drop=True)
remaining_scaled = remaining_scaled.reset_index(drop=True)

# Concatenate the first column back with the scaled remaining columns
syd = pd.concat([first_col, remaining_scaled], axis=1)

# Print the final DataFrame with the unscaled first column and scaled remaining columns
print(syd)


      income  surface_structure_coherence_mean  \
0         31                          0.734941   
1        110                          0.830092   
2        176                          0.482333   
3        240                          0.767768   
4        380                          0.651340   
...      ...                               ...   
9411    4513                          0.711447   
9412    4523                          0.548892   
9413    4596                          0.675843   
9414    4666                          0.552867   
9415    4750                          0.651223   

      surface_structure_disorderliness_mean  \
0                                  0.300280   
1                                  0.399101   
2                                  0.155683   
3                                  0.226679   
4                                  0.415478   
...                                     ...   
9411                               0.344419   
9412                   

In [8]:
# Normalising predictor variables

first_col = preds.iloc[:, 0]
remaining_cols = preds.iloc[:, 1:]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the remaining columns of X and transform them
remaining_scaled = pd.DataFrame(scaler.fit_transform(remaining_cols), columns=remaining_cols.columns)

# Reset indices to ensure they align for concatenation
first_col = first_col.reset_index(drop=True)
remaining_scaled = remaining_scaled.reset_index(drop=True)

# Concatenate the first column back with the scaled remaining columns
preds = pd.concat([first_col, remaining_scaled], axis=1)

# Print the final DataFrame with the unscaled first column and scaled remaining columns
print(preds)


        id  surface_structure_coherence_mean  \
0      505                          0.734941   
1      632                          0.830092   
2      432                          0.482333   
3      798                          0.767768   
4     1667                          0.651340   
...    ...                               ...   
9411  9205                          0.711447   
9412  8959                          0.548892   
9413  9670                          0.675843   
9414  9269                          0.552867   
9415  9690                          0.651223   

      surface_structure_disorderliness_mean  \
0                                  0.300280   
1                                  0.399101   
2                                  0.155683   
3                                  0.226679   
4                                  0.415478   
...                                     ...   
9411                               0.344419   
9412                               0.300637   


In [9]:
# Creating X/y dataframes
y = syd.income
X = preds.drop('id', axis=1)

In [10]:
# Dropping geographic coordinates
X = X.drop('x_coord', axis=1)
X = X.drop('y_coord', axis=1)

In [11]:
column_names = X.columns.tolist
column_names

<bound method IndexOpsMixin.tolist of Index(['surface_structure_coherence_mean',
       'surface_structure_disorderliness_mean',
       'surface_roughness_disorderliness_mean', 'colour_saturation_mean',
       'colour_hue_coherence_mean', 'colour_hue_disorderliness_mean',
       'tcap_wetness_mean', 'tcap_greenness_std', 'nir_band_mean',
       'colour_brightness_coherence_mean',
       'colour_brightness_disorderliness_mean', 'ndwi_wetness_contrast_mean',
       'ndwi_wetness_coherence_std', 'tcap_wetness_disorderliness_mean',
       'tcap_built_coherence_std'],
      dtype='object')>

In [12]:

# Split the subsetted data into training and validation sets
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
dataset_dir = './inc_syd'
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

# We will be able to work with files with/without header and
# with different separators.

syd.to_csv(
    os.path.join(dataset_dir, 'train.csv'),
    index=False, sep=',', header=True
)
preds.to_csv(
    os.path.join(dataset_dir, 'preds.csv'),
    index=False, sep=',', header=True
)

In [14]:
pool1 = Pool(data=X, label=y)

print('Dataset shape')
print('dataset 1:' + str(pool1.shape))

print('\n')
print('Column names')
print('dataset 1:')
print(pool1.get_feature_names()) 

Dataset shape
dataset 1:(9416, 15)


Column names
dataset 1:
['surface_structure_coherence_mean', 'surface_structure_disorderliness_mean', 'surface_roughness_disorderliness_mean', 'colour_saturation_mean', 'colour_hue_coherence_mean', 'colour_hue_disorderliness_mean', 'tcap_wetness_mean', 'tcap_greenness_std', 'nir_band_mean', 'colour_brightness_coherence_mean', 'colour_brightness_disorderliness_mean', 'ndwi_wetness_contrast_mean', 'ndwi_wetness_coherence_std', 'tcap_wetness_disorderliness_mean', 'tcap_built_coherence_std']


TBB Warning: The number of workers is currently limited to 5. The request for 39 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



In [16]:
# Optimization of hyperparameters
train_pool = Pool(data=X_train, label=y_train)
validation_pool = Pool(data=X_validation, label=y_validation)

# Define the function to optimize
def catboost_cv(learning_rate, iterations, depth, l2_leaf_reg):
    # Convert continuous parameters to integer where necessary
    iterations = int(iterations)
    depth = int(depth)
    
    # Define and train the model
    model = CatBoostRegressor(
        learning_rate=learning_rate,
        iterations=iterations,
        depth=depth,
        l2_leaf_reg=l2_leaf_reg,
        loss_function='RMSE',
        eval_metric='R2',
        random_seed=42,
        logging_level='Silent'
    )
    
    model.fit(train_pool, eval_set=validation_pool, use_best_model=True)
    
    # Calculate R-squared values
    train_r2 = model.score(train_pool)
    test_r2 = model.score(validation_pool)
    
    # Ensure that train R-squared does not exceed test R-squared by more than 0.1
    if train_r2 - test_r2 > 0.1:
        return 0  # Penalize the function if the condition is not met
    
    return test_r2

# Define the initial bounds for hyperparameters
param_bounds = {
    'learning_rate': (0.01, 0.05), 
    'iterations': (1000, 10000), 
    'depth': (2, 4), 
    'l2_leaf_reg': (2, 100)
}

# Initialize Bayesian Optimization
optimizer = BayesianOptimization(
    f=catboost_cv,
    pbounds=param_bounds,
    random_state=42,
    verbose=2
)

# Perform optimization
optimizer.maximize(
    init_points=10,  # Number of initial random points
    n_iter=30       # Number of optimization iterations
)

# Extract the best parameters
best_params = optimizer.max['params']
best_params['iterations'] = int(best_params['iterations'])
best_params['depth'] = int(best_params['depth'])

print("Best hyperparameters found were:")
print(best_params)


|   iter    |  target   |   depth   | iterat... | l2_lea... | learni... |
-------------------------------------------------------------------------
| 1         | 0.0       | 2.749     | 9.556e+03 | 73.74     | 0.03395   |
| 2         | 0.0       | 2.312     | 2.404e+03 | 7.692     | 0.04465   |
| 3         | 0.0       | 3.202     | 7.373e+03 | 4.017     | 0.0488    |
| 4         | 0.0       | 3.665     | 2.911e+03 | 19.82     | 0.01734   |
| 5         | 0.0       | 2.608     | 5.723e+03 | 44.33     | 0.02165   |
| 6         | 0.0       | 3.224     | 2.255e+03 | 30.63     | 0.02465   |
| 7         | 0.0       | 2.912     | 8.067e+03 | 21.57     | 0.03057   |
| 8         | 0.3892    | 3.185     | 1.418e+03 | 61.54     | 0.01682   |
| 9         | 0.0       | 2.13      | 9.54e+03  | 96.63     | 0.04234   |
| 10        | 0.3774    | 2.609     | 1.879e+03 | 69.05     | 0.02761   |
| 11        | 0.0       | 3.36      | 1.65e+03  | 5.004     | 0.02031   |
| 12        | 0.3314    | 2.293     | 

In [18]:
# Fitting CatBoost model
best_model = CatBoostRegressor(
    random_seed=63,
    iterations=1257,
    task_type="CPU",
    learning_rate=0.03449758553882252,
    l2_leaf_reg = 58,
    depth = 3,
    loss_function='MAE'
)
best_model.fit(
    X_train, y_train,
    verbose=False,
    eval_set=(X_validation, y_validation),
    early_stopping_rounds=50,
    plot=False
)
results = best_model.get_evals_result()

In [20]:
pool1 = Pool(data=X, label=y)

print('Dataset shape')
print('dataset 1:' + str(pool1.shape))

print('\n')
print('Column names')
print('dataset 1:')
print(pool1.get_feature_names()) 

Dataset shape
dataset 1:(9416, 15)


Column names
dataset 1:
['surface_structure_coherence_mean', 'surface_structure_disorderliness_mean', 'surface_roughness_disorderliness_mean', 'colour_saturation_mean', 'colour_hue_coherence_mean', 'colour_hue_disorderliness_mean', 'tcap_wetness_mean', 'tcap_greenness_std', 'nir_band_mean', 'colour_brightness_coherence_mean', 'colour_brightness_disorderliness_mean', 'ndwi_wetness_contrast_mean', 'ndwi_wetness_coherence_std', 'tcap_wetness_disorderliness_mean', 'tcap_built_coherence_std']


In [21]:
# Get parameters from the best model but exclude 'loss_function' if it's already set
params = {key: val for key, val in best_model.get_params().items() if key != 'loss_function'}

# Define the model with optimal parameters and explicitly set the loss function and custom metric
model = CatBoostRegressor(loss_function='MAE', custom_metric='R2', **params)

# Setup Repeated K-Fold cross-validation
rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# Prepare lists to store results
train_r2_results = []
test_r2_results = []
train_mae_results = []
test_mae_results = []
train_rmse_results = []
test_rmse_results = []

# Loop over each fold
for train_index, test_index in rkf.split(X):
    train_pool = Pool(X.iloc[train_index], y[train_index])
    test_pool = Pool(X.iloc[test_index], y[test_index])

    # Fit model
    model.fit(train_pool, eval_set=test_pool, verbose=False)

    # Evaluate on the training set
    train_predictions = model.predict(train_pool)
    train_r2 = model.score(train_pool.get_features(), train_pool.get_label())
    train_mae = mean_absolute_error(train_pool.get_label(), train_predictions)
    train_rmse = np.sqrt(mean_squared_error(train_pool.get_label(), train_predictions))
    train_r2_results.append(train_r2)
    train_mae_results.append(train_mae)
    train_rmse_results.append(train_rmse)

    # Evaluate on the testing set
    test_predictions = model.predict(test_pool)
    test_r2 = model.score(test_pool.get_features(), test_pool.get_label())
    test_mae = mean_absolute_error(test_pool.get_label(), test_predictions)
    test_rmse = np.sqrt(mean_squared_error(test_pool.get_label(), test_predictions))
    test_r2_results.append(test_r2)
    test_mae_results.append(test_mae)
    test_rmse_results.append(test_rmse)

# Calculate the average and standard deviation of R-squared across all train and test folds
mean_train_r2 = np.mean(train_r2_results)
std_train_r2 = np.std(train_r2_results)
mean_test_r2 = np.mean(test_r2_results)
std_test_r2 = np.std(test_r2_results)

# Calculate the average and standard deviation of MAE across all train and test folds
mean_train_mae = np.mean(train_mae_results)
std_train_mae = np.std(train_mae_results)
mean_test_mae = np.mean(test_mae_results)
std_test_mae = np.std(test_mae_results)

# Calculate the average and standard deviation of RMSE across all train and test folds
mean_train_rmse = np.mean(train_rmse_results)
std_train_rmse = np.std(train_rmse_results)
mean_test_rmse = np.mean(test_rmse_results)
std_test_rmse = np.std(test_rmse_results)

print("Average Train R-squared:", mean_train_r2)
print("Train R-squared Standard Deviation:", std_train_r2)
print("Average Test R-squared:", mean_test_r2)
print("Test R-squared Standard Deviation:", std_test_r2)
print("Average Train MAE:", mean_train_mae)
print("Train MAE Standard Deviation:", std_train_mae)
print("Average Test MAE:", mean_test_mae)
print("Test MAE Standard Deviation:", std_test_mae)
print("Average Train RMSE:", mean_train_rmse)
print("Train RMSE Standard Deviation:", std_train_rmse)
print("Average Test RMSE:", mean_test_rmse)
print("Test RMSE Standard Deviation:", std_test_rmse)


Average Train R-squared: 0.4807757578891862
Train R-squared Standard Deviation: 0.003080930452702845
Average Test R-squared: 0.41494470205656747
Test R-squared Standard Deviation: 0.015067033925977887
Average Train MAE: 348.60142280924026
Train MAE Standard Deviation: 1.5502784672720262
Average Test MAE: 382.91277700142933
Test MAE Standard Deviation: 6.096746548373658
Average Train RMSE: 487.11443222030374
Train RMSE Standard Deviation: 2.1258364048019804
Average Test RMSE: 516.7360144402118
Test RMSE Standard Deviation: 8.972346632159105
